In [ ]:
import networkx as nx 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sys
import seaborn as sns
import json
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
from nltk.tokenize import sent_tokenize, word_tokenize
from wordcloud import WordCloud, STOPWORDS

#This section imports all libraries to be used

data1 = pd.read_json(
                     "https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/I?page=1&api_key=RGAPI-ef189c75-a0a1-46d0-a3bb-ad56d2820372"
                  )
data2 = pd.read_json(
                     "https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/I?page=2&api_key=RGAPI-ef189c75-a0a1-46d0-a3bb-ad56d2820372"
                  )
data3 = pd.read_json(
    'https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/I?page=3&api_key=RGAPI-ef189c75-a0a1-46d0-a3bb-ad56d2820372'
)

data4 = pd.read_json(
                     "https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/IV?page=1&api_key=RGAPI-ef189c75-a0a1-46d0-a3bb-ad56d2820372"
                  )
data5 = pd.read_json(
                     "https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/IV?page=2&api_key=RGAPI-ef189c75-a0a1-46d0-a3bb-ad56d2820372"
                  )
data6 = pd.read_json(
    'https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/IV?page=3&api_key=RGAPI-ef189c75-a0a1-46d0-a3bb-ad56d2820372'
)

#This accesses the API for all data

d1 = pd.DataFrame(data1)
d2 = pd.DataFrame(data2)
d3 = pd.DataFrame(data3)

d4 = pd.DataFrame(data4)
d5 = pd.DataFrame(data5)
d6 = pd.DataFrame(data6)

silver_frames = [d1, d2, d3]
gold_frames = [d4, d5, d6]

silver = pd.concat(silver_frames, ignore_index=True)
gold = pd.concat(gold_frames, ignore_index=True)

#this puts the data into the two dataframes, silver and gold, based on the different ranks



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
#silver = silver.drop(['miniSeries'], axis=1)
silver = silver.drop_duplicates(['summonerId'])
silver.describe()
gold.describe()

#this section does basic preprocessing, dropping a column with NaN values, removing duplicates, and checking to make sure no outliers or bias are included

total_new = [gold, silver]
total = pd.concat(total_new, ignore_index=True)
total
print(total.iloc[:3])

total.to_csv('riot_data.csv')
#from google.colab import files
#files.download("riot_data.csv")

#this last part combines the ranks into one dataframe, converts the frame to csv and exports it to my local drive

total

                               leagueId        queueType  ... hotStreak miniSeries
0  19d45391-e9da-426a-9995-e621196acb1d  RANKED_SOLO_5x5  ...     False        NaN
1  38a07281-4d75-48d9-a651-61e17764d163  RANKED_SOLO_5x5  ...     False        NaN
2  323a185d-842e-4459-8ce8-a5bbe2e5e777  RANKED_SOLO_5x5  ...      True        NaN

[3 rows x 14 columns]


,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,miniSeries
0,19d45391-e9da-426a-9995-e621196acb1d,RANKED_SOLO_5x5,GOLD,IV,kVxfaRQ-Bd7_T960_71vxqHpzyNskri7DpjRab9tlmDxaeRz,ryonoon,0,110,83,False,False,False,False,NaN
1,38a07281-4d75-48d9-a651-61e17764d163,RANKED_SOLO_5x5,GOLD,IV,AmEWdiCcCsgjFOc5WMnyNskLYaQmEX0x5rM2GCcTJc7RKSFH,ComsicGhostRider,0,150,144,False,False,False,False,NaN
2,323a185d-842e-4459-8ce8-a5bbe2e5e777,RANKED_SOLO_5x5,GOLD,IV,kRsOIhIiwCpLfImVNykWxPSWDvnkKo7UfFF06bmpSJdyXAUC,Ichigodayo,92,142,156,False,False,False,True,NaN
3,6603e0f0-3bd0-4d92-91af-2cfb06ff4903,RANKED_SOLO_5x5,GOLD,IV,KpAMw6FsmnqUe5E__yoESRRPykjvUcBSvc8Rum4ahemlJ_eE,S1apNutz,0,50,42,False,False,False,False,NaN
4,49be3da0-d0a0-4a5b-b8dd-35b86bd05c0e,RANKED_SOLO_5x5,GOLD,IV,GdmxApY_JJCmPOHzDRR28ixuGFivfNvQnYnWviadgrfZTQY2,big grammy,13,185,208,False,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,eb4950dd-48b8-427c-acdd-6b0e68d84645,RANKED_SOLO_5x5,SILVER,I,52ZSFyApZ3n0r0hX1ZyTvKCtLZ6ILFOcJ5VQZv56cCG06kGc,Gìo,61,298,341,False,False,True,False,NaN
1216,797153da-1b0b-4eb0-9945-e9de536e27dc,RANKED_SOLO_5x5,SILVER,I,FKEgooP9zu433leiwYiwlWMmdOPWbxOMyfCE2P_xqvX12jn0,GabbyLol,46,406,400,False,False,True,False,NaN
1217,42f8ebc4-8cdb-46eb-b825-e424ea25ea30,RANKED_SOLO_5x5,SILVER,I,T5QuEBKqOV3tR5c9uxLctbwlNGay73uqFRMrmsgW6leNPNqh,glicoglico,85,80,78,False,False,False,False,NaN
1218,06902269-7288-4a04-b7ad-0db9cad93aa7,RANKED_SOLO_5x5,SILVER,I,Oulq8qjsnYHvKSqDNzgfrhTRbUybb49w804eMVovdwr5X3PZ,VuseOriginal,74,43,42,False,False,False,False,NaN


In [ ]:
text = total.head().to_json()
print(word_tokenize(text))
#this section converts the first 5 rows of the dataframe back to json to use word tokenizers
#it then prints the tokens created

['{', '``', 'leagueId', "''", ':', '{', '``', '0', "''", ':', "''", '19d45391-e9da-426a-9995-e621196acb1d', "''", ',', "''", '1', "''", ':', "''", '38a07281-4d75-48d9-a651-61e17764d163', "''", ',', "''", '2', "''", ':', "''", '323a185d-842e-4459-8ce8-a5bbe2e5e777', "''", ',', "''", '3', "''", ':', "''", '6603e0f0-3bd0-4d92-91af-2cfb06ff4903', "''", ',', "''", '4', "''", ':', "''", '49be3da0-d0a0-4a5b-b8dd-35b86bd05c0e', "''", '}', ',', "''", 'queueType', "''", ':', '{', '``', '0', "''", ':', "''", 'RANKED_SOLO_5x5', "''", ',', "''", '1', "''", ':', "''", 'RANKED_SOLO_5x5', "''", ',', "''", '2', "''", ':', "''", 'RANKED_SOLO_5x5', "''", ',', "''", '3', "''", ':', "''", 'RANKED_SOLO_5x5', "''", ',', "''", '4', "''", ':', "''", 'RANKED_SOLO_5x5', "''", '}', ',', "''", 'tier', "''", ':', '{', '``', '0', "''", ':', "''", 'GOLD', "''", ',', "''", '1', "''", ':', "''", 'GOLD', "''", ',', "''", '2', "''", ':', "''", 'GOLD', "''", ',', "''", '3', "''", ':', "''", 'GOLD', "''", ',', "''", '4', "

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer


q = total.drop(['leaguePoints', 'wins', 'losses', 'leagueId', 'summonerId', 'miniSeries'], axis=1).to_json()

stop_words = nltk.corpus.stopwords.words("english")

word_vectorizer = CountVectorizer(
    ngram_range=(1,3), 
    stop_words = stop_words,
    analyzer='word')
sparse_matrix = word_vectorizer.fit_transform([q])
frequencies = sum(sparse_matrix).toarray()[0]
ngram_df = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])
ngram_df.nlargest(20, 'frequency')

# This section converts the total dataframe back to json for processing, then finds the 20 most frequently occuring unigrams, bigrams and trigrams

,frequency
false,4686
ranked_solo_5x5,1220
iv,617
gold,615
silver,605
true,194
false false,34
false false false,29
100,9
11,9


In [ ]:
fd=nltk.FreqDist(text)
from nltk.sentiment import SentimentIntensityAnalyzer 
sia = SentimentIntensityAnalyzer()

sia.polarity_scores(q)
g = silver.drop(['leaguePoints', 'wins', 'losses', 'leagueId', 'summonerId', 'miniSeries'], axis=1).to_json()
h = gold.drop(['leaguePoints', 'wins', 'losses', 'leagueId', 'summonerId'], axis=1).to_json()
sia.polarity_scores(g)
sia.polarity_scores(h)

# In this section I use the sentiment analyzer on the text of the total dataframe, and then separate them based on rank
# Because I don't have dates as data, I thought separating/grouping them by a different variable might suffice
# The analysis suggests that, while the data for gold rank is mainly neutral with some of it trending towards the positive, that silver tends to be a bit more negative. The main difference between the two datasets are... 
# ...the summoner names. It may suggest the lower rank has more negative names.

{'compound': 0.5812, 'neg': 0.02, 'neu': 0.943, 'pos': 0.037}

from google.colab import files
uploaded = files.upload()

thats what i need to upload files to colab

from google.colab import files
files.download("data.csv")

thats what i need to download files from colab

'{"queueType":{"0":"RANKED_SOLO_5x5","1":"RANKED_SOLO_5x5","2":"RANKED_SOLO_5x5","3":"RANKED_SOLO_5x5","4":"RANKED_SOLO_5x5","5":"RANKED_SOLO_5x5","6":"RANKED_SOLO_5x5","7":"RANKED_SOLO_5x5","8":"RANKED_SOLO_5x5","9":"RANKED_SOLO_5x5","10":"RANKED_SOLO_5x5","11":"RANKED_SOLO_5x5","12":"RANKED_SOLO_5x5","13":"RANKED_SOLO_5x5","14":"RANKED_SOLO_5x5","15":"RANKED_SOLO_5x5","16":"RANKED_SOLO_5x5","17":"RANKED_SOLO_5x5","18":"RANKED_SOLO_5x5","19":"RANKED_SOLO_5x5","20":"RANKED_SOLO_5x5","21":"RANKED_SOLO_5x5","22":"RANKED_SOLO_5x5","23":"RANKED_SOLO_5x5","24":"RANKED_SOLO_5x5","25":"RANKED_SOLO_5x5","26":"RANKED_SOLO_5x5","27":"RANKED_SOLO_5x5","28":"RANKED_SOLO_5x5","29":"RANKED_SOLO_5x5","30":"RANKED_SOLO_5x5","31":"RANKED_SOLO_5x5","32":"RANKED_SOLO_5x5","33":"RANKED_SOLO_5x5","34":"RANKED_SOLO_5x5","35":"RANKED_SOLO_5x5","36":"RANKED_SOLO_5x5","37":"RANKED_SOLO_5x5","38":"RANKED_SOLO_5x5","39":"RANKED_SOLO_5x5","40":"RANKED_SOLO_5x5","41":"RANKED_SOLO_5x5","42":"RANKED_SOLO_5x5","43":"